In [1]:
%%javascript
var rate = 0;
// apply setting to  all current CodeMirror instances
IPython.notebook.get_cells().map(
    function(c) {  return c.code_mirror.options.cursorBlinkRate=rate;  }
);

// make sure new CodeMirror instance also use this setting
CodeMirror.defaults.cursorBlinkRate=rate;

<IPython.core.display.Javascript object>

In [2]:
from collections import OrderedDict
#from svcaller.effect.consequence import *

In [3]:
import pandas as pd

In [4]:
test_DEL_1 = pd.DataFrame(
    OrderedDict({
    "chrom": [1],
    "start": [700],
    "end": [800],
    "type": ["DEL"],
    "score": [0],
    "strand": ["+"]
})).iloc[0,:]

test_DELs = pd.DataFrame(
    OrderedDict({
    "chrom": [1, 1],
    "start": [700, 1000],
    "end": [800, 1100],
    "type": ["DEL", "DEL"],
    "score": [0, 0],
    "strand": ["+", "+"]
}))

test_DUPs = pd.DataFrame(
    OrderedDict({
    "chrom": [1, 1],
    "start": [700, 1000],
    "end": [800, 1100],
    "type": ["DUP", "DUP"],
    "score": [0, 0],
    "strand": ["+", "+"]
}))

test_regions_1 = pd.DataFrame(
    OrderedDict({
    "chrom": [1, 1],
    "start": [500, 1500],
    "end": [1000, 2000],
    "gene": ["TS1", "TS1"]
}))

In [5]:
from svcaller.calling.events import clust_filt

In [6]:
import pysam
input_bam = "/Users/thowhi/vms/fixing/fairbanks/event_filtered.bam"
samfile = pysam.AlignmentFile(input_bam, "rb")
reads = [r for r in list(samfile)]

In [7]:
??clust_filt

In [15]:
reads_subset = reads[:500000]

In [18]:
reads_subset[-1].rname

22

In [16]:
#clust_filt(reads_subset)
%prun -s cumulative clust_filt(reads_subset)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [6]:
from svcaller.calling.events import read_sv_gtf, SvType

In [107]:
sv_gtf_file = open("/Users/thowhi/analysis/scratch/tmp-events.gtf")
names = ["chrom", "source", "feature", "start", "end", "score", "strand", "frame", "attribute"]
gtf_file_df = pd.read_table(sv_gtf_file, sep="\t", header=None, index_col=None, names=names)
gtf_file_termini = gtf_file_df.loc[gtf_file_df["feature"] == "exon", :].copy()
event_names = gtf_file_termini["attribute"].apply(
    lambda attribute: attribute.split(" ")[1].replace("\"", "").replace(";", ""))
gtf_file_termini["event_names"] = event_names
groups = gtf_file_termini.groupby("event_names")

DEBUG:past.translation:Running find_module: pandas._libs.operator...
DEBUG:past.translation:Py2Fixer could not find {0}
DEBUG:past.translation:Exception was: operator)


In [27]:
def extract_bed_data(group, event_type):
    assert len(group) == 2
    bed_data = []
    if event_type == SvType.TRA:
        # Represent each terminus as a separate bed item:
        for _, row in group.iterrows():
            bed_data.append([row[0], row[3], row[4], event_type.value, row[5], row[6]])
    else:
        coords = group.iloc[:,3:5].values.flatten()
        start = min(coords)
        end = max(coords)
        strand = None
        if group.iloc[0,6] == group.iloc[1,6]:
            strand = group.iloc[0,6]
        bed_data.append([group.iloc[0,0], start, end, event_type.value, group.iloc[0,5], strand])
    return bed_data


def read_sv_gtf(sv_gtf_file, event_type):
    """
    Read a structural variants gtf file and generate a bed-format compatible table
    representing the event coordinates.

    :param sv_gtf_file: An open gtf-formatted file containing structural variant calls
    :param event_type: An SvType enumeration value specifying the structural variant type
    :return: A data frame containing the structural variant calls
    """

    names = ["chrom", "source", "feature", "start", "end", "score", "strand", "frame", "attribute"]
    gtf_file_df = pd.read_table(sv_gtf_file, sep="\t", header=None, index_col=None, names=names)
    gtf_file_termini = gtf_file_df.loc[gtf_file_df["feature"] == "exon", :].copy()
    event_names = gtf_file_termini["attribute"].apply(
        lambda attribute: attribute.split(" ")[1].replace("\"", "").replace(";", ""))
    gtf_file_termini["event_names"] = event_names
    groups = gtf_file_termini.groupby("event_names")
    def bed_extractor(group):
        return extract_bed_data(group, event_type)

    return groups.apply(bed_extractor)

In [38]:
x = read_sv_gtf(open("/Users/thowhi/analysis/scratch/test1.gtf"), SvType.TRA)

In [39]:
pd.DataFrame(x[1])

,0,1,2,3,4,5
0,21,39874777,39875065,TRA,46,+
1,21,42871970,42872366,TRA,46,+


In [42]:
with open("TumourSupressors.bed", 'w') as ts_file:
    ts_file.write('''1\t500\t1000\tTS1
1\t1500\t2000\tTS1
2\t500\t1000\tTS2
2\t1500\t2000\tTS2
''')

with open("AR.bed", 'w') as ar_file:
    ar_file.write('''X\t500\t1000\tTS1
X\t1500\t2000\tTS1
X\t2500\t3000\tTS2
X\t3500\t4000\tTS2
''')

with open("FusionCandidates.bed", 'w') as fusion_file:
    fusion_file.write('''21\t1000000\t1100000\tERG
21\t2000000\t2100000\tTMPRSS2
''')

with open("SVs.bed", 'w') as svs_file:
    svs_file.write('''21\t1500000\t1500100\tDEL\t0\t+
21\t2500000\t2500100\tDEL\t0\t+
1\t1600\t1700\tDEL\t0\t+
1\t1800\t1900\tTRA\t0\t+
1\t1400\t1600\tDEL\t0\t+
1\t3100\t3200\tDEL\t0\t+
''')

In [128]:
predict_del_effect(test_sv, GeneClass.TUMOUR_SUPRESSOR, test_regions)

<SvEffect.NO_OVERLAP: 'NO_OVERLAP'>

In [67]:
svs_table = extract_groups('SVs.bed')
ts_table = extract_groups('TumourSupressors.bed')

predict_svs_effect_for_class(svs_table, GeneClass.TUMOUR_SUPRESSOR, ts_table)

TRACE: sv_type: SvType.DEL
TRACE: sv_row:
0       1
1    1600
2    1700
3     DEL
4       0
5       +
Name: 2, dtype: object
TRACE: sv_row:
0       1
1    1400
2    1600
3     DEL
4       0
5       +
Name: 4, dtype: object
TRACE: sv_type: SvType.INV
TRACE: sv_type: SvType.TRA
TRACE: sv_row:
0       1
1    1800
2    1900
3     TRA
4       0
5       +
Name: 3, dtype: object
TRACE: sv_type: SvType.DUP
TRACE: sv_type: SvType.DEL
TRACE: sv_type: SvType.INV
TRACE: sv_type: SvType.TRA
TRACE: sv_type: SvType.DUP


defaultdict(list, {'TS1': [None, None, None]})